# Final Project for Research Computing in Earth Science
## *Investigating Ocean Acidification in the Arctic* 

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pooch
import matplotlib
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'

# CESM Emission Scenarios

[Index of /data/oceans/ncei/ocads/data/0259391/nc/ESMs/CESM2]('https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0259391/nc/ESMs/CESM2/')

In [7]:
# structure : {base_url}{variables}{scenarios}
cesm_base_url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0259391/nc/ESMs/CESM2/CESM2_{}_{}.nc'
variables = ['Temperature','Salinity','pHT','pCO2']
scenarios = ['historical','ssp126','ssp245','ssp370','ssp585']

cesm_url_list = []
for i in variables:
    for j in scenarios:
        cesm_url_list.append(cesm_base_url.format(i,j)) 

#cesm_url_list

In [ ]:
cesm = xr.open_mfdataset(cesm_url_list)

# Sea Ice Extent

[Sea Ice Concentration, NOAA/NSIDC Climate Data Record V3, Arctic, 25km, Science Quality, 1978-2019, Monthly DEPRECATED]('https://polarwatch.noaa.gov/erddap/files/nsidcCDRiceSQnhmday/')

## Sea Ice Concentration

In [4]:
ice_base_url = 'https://polarwatch.noaa.gov/erddap/files/nsidcCDRiceSQnhmday/seaice_conc_monthly_nh_{}_{}_v03r01.nc'

yearmonthn07 = pd.period_range(start='197811', end='198707', freq='m').strftime('%Y%m').tolist()
yearmonthf08 = pd.period_range(start='198708', end='199112', freq='m').strftime('%Y%m').tolist()
yearmonthf11 = pd.period_range(start='199201', end='199509', freq='m').strftime('%Y%m').tolist()
yearmonthf13 = pd.period_range(start='199510', end='200712', freq='m').strftime('%Y%m').tolist()
yearmonthf17 = pd.period_range(start='200801', end='201912', freq='m').strftime('%Y%m').tolist()

ice_url_list = []    
for j in yearmonthn07:
    ice_url_list.append(ice_base_url.format('n07',j)) 
for j in yearmonthf08:
    ice_url_list.append(ice_base_url.format('f08',j))   
for j in yearmonthf11:
    ice_url_list.append(ice_base_url.format('f11',j))     
for j in yearmonthf13:
    ice_url_list.append(ice_base_url.format('f13',j)) 
for j in yearmonthf17:
    ice_url_list.append(ice_base_url.format('f17',j)) 
    
#ice_url_list

In [6]:
#url = 'https://polarwatch.noaa.gov/erddap/files/nsidcCDRiceSQsh1day/2017/seaice_conc_daily_sh_f17_20170807_v03r01.nc'
fname = pooch.retrieve(ice_url_list,known_hash=None)
ds_ice = xr.open_mfdataset(fname) #, drop_variables='melt_onset_day_seaice_conc_cdr'

AttributeError: 'list' object has no attribute 'encode'

In [48]:
ice = xr.open_mfdataset(ice_url_list)

syntax error, unexpected $end, expecting ';'
context: Error { code=404; message="Not Found: File not found: seaice_conc_monthly_nh_n07_197811_v03r01.nc.dds .";}^


OSError: [Errno -90] NetCDF: file not found: b'https://polarwatch.noaa.gov/erddap/files/nsidcCDRiceSQnhmday/seaice_conc_monthly_nh_n07_197811_v03r01.nc'

In [52]:
test_url = 'https://polarwatch.noaa.gov/erddap/files/nsidcCDRiceSQnhmday/seaice_conc_monthly_nh_n07_197811_v03r01.nc'

In [54]:
test = xr.open_dataset(test_url)

syntax error, unexpected $end, expecting ';'
context: Error { code=404; message="Not Found: File not found: seaice_conc_monthly_nh_n07_197811_v03r01.nc.dds .";}^


OSError: [Errno -90] NetCDF: file not found: b'https://polarwatch.noaa.gov/erddap/files/nsidcCDRiceSQnhmday/seaice_conc_monthly_nh_n07_197811_v03r01.nc'